In [48]:
import nltk
import csv
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import ( BernoulliNB,ComplementNB,MultinomialNB)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import metrics

nltk.download(["names","stopwords","averaged_perceptron_tagger","vader_lexicon","punkt"])

[nltk_data] Downloading package names to C:\Users\MSI
[nltk_data]     GP63\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\MSI
[nltk_data]     GP63\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MSI GP63\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to C:\Users\MSI
[nltk_data]     GP63\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\MSI
[nltk_data]     GP63\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
tweets = []
testTweets = []
#Data understanding & reading
with open('dreaddit-train.csv',encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            tweets.append([row[0],row[3],row[5]])
        line_count += 1
    print(f'Processed {line_count} lines for training.')
with open('dreaddit-test.csv',encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            testTweets.append([row[1],row[4],row[5]])
        line_count += 1
    print(f'Processed {line_count} lines for testing.')

Processed 2839 lines for training.
Processed 716 lines for testing.


In [41]:
i = 0
added = 0
subreddits = {}
sia = SentimentIntensityAnalyzer()
numericElements = []
numericElements2 = []
labels = []
for tweet in tweets:
    polarity = sia.polarity_scores(tweet[1])
    if tweet[0] not in subreddits:
        subreddits[tweet[0]] = added
        added = added + 1
    numericElements.append([polarity["neg"],polarity["pos"],polarity["neu"],polarity["compound"]])
    del(polarity["compound"])
    numericElements2.append(polarity)
    i = i + 1
    labels.append(tweet[2])
print(numericElements[0])
print(subreddits)
print(numericElements2[0])

[0.176, 0.093, 0.731, -0.8551]
{'ptsd': 0, 'assistance': 1, 'relationships': 2, 'survivorsofabuse': 3, 'domesticviolence': 4, 'anxiety': 5, 'homeless': 6, 'stress': 7, 'almosthomeless': 8, 'food_pantry': 9}
{'neg': 0.176, 'neu': 0.731, 'pos': 0.093}


In [50]:
numericElementsTest = []
numericElementsTest2 = []
labelsTest = []
for tweet in testTweets:
    polarity = sia.polarity_scores(tweet[1])
    numericElementsTest.append([polarity["neg"],polarity["pos"],polarity["neu"],polarity["compound"]])
    labelsTest.append(tweet[2])
    del(polarity["compound"])
    numericElementsTest2.append(polarity)
print(numericElementsTest[0])

[0.078, 0.186, 0.736, 0.6908]


In [ ]:
i = 0
for tweet in tweets:
    catDictionary = {}
    for cat in subreddits.keys():
        sameCat = cat == tweet[0]
        if sameCat:
            numericElements[i].append(1)
            catDictionary[cat] = 1
        else :
            numericElements[i].append(0)
            catDictionary[cat] = 0
    numericElements2[i] = (dict(catDictionary, **numericElements2[i]),tweet[2])
    i = i + 1
print(numericElements[0])
print(numericElements2[0])

In [51]:
i = 0
for tweet in testTweets:
    catDictionary = {}
    for cat in subreddits.keys():
        sameCat = cat == tweet[0]
        if sameCat:
            numericElementsTest[i].append(1)
            catDictionary[cat] = 1
        else :
            numericElementsTest[i].append(0)
            catDictionary[cat] = 0
    numericElementsTest2[i] = (dict(catDictionary,**numericElementsTest2[i]),tweet[2])
    i = i + 1
print(numericElementsTest[0])

[0.078, 0.186, 0.736, 0.6908, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [64]:
model = KNeighborsClassifier(n_neighbors=23, algorithm = 'brute')
model.fit(numericElements,labels)

KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=23, p=2,
                     weights='uniform')

In [67]:
y_test = model.predict(numericElementsTest)
accuracy = metrics.accuracy_score(y_test, labelsTest)
precision = metrics.precision_score(y_test, labelsTest, pos_label='1')
recall = metrics.recall_score(y_test, labelsTest, pos_label='1')
print(F"Accuracy: {accuracy:.2f}")
print(F"Precision: {precision:.2f}")
print(F"Recall: {recall:.2f}")

Accuracy: 0.71
Precision: 0.75
Recall: 0.70


In [55]:
classifiers = {
    "BernoulliNB": BernoulliNB(),
    "KNeighborsClassifier": KNeighborsClassifier(n_neighbors=23),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier(),
}
for name, sklearn_classifier in classifiers.items():
    classifier = nltk.classify.SklearnClassifier(sklearn_classifier)
    classifier.train(numericElements2)
    accuracy = nltk.classify.accuracy(classifier, numericElementsTest2)
    print(F"{accuracy:.2%} - {name}")

62.66% - BernoulliNB
69.65% - KNeighborsClassifier
62.52% - DecisionTreeClassifier
65.87% - RandomForestClassifier
69.79% - LogisticRegression
70.35% - MLPClassifier
70.21% - AdaBoostClassifier
